<a href="https://colab.research.google.com/github/ufbfung/mimic-fhir/blob/main/Exploring_mimic_iv_data_on_fhir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview
In this notebook, I will be exploring the mimic IV demo database on FHIR and perhaps create some interesting functions in python for it.

In [24]:
import pandas as pd
import json

# Function to parse a single patient record from FHIR format
def parse_patient_record(record):
    patient_data = {
        'id': record['id'],
        'gender': record['gender'],
        'birthDate': record['birthDate'],
        'maritalStatus': record.get('maritalStatus', {}).get('coding', [{}])[0].get('code', None),
        'race': None,
        'ethnicity': None,
        'birthSex': None
    }

    # Parsing extensions for race, ethnicity, and birthSex
    for extension in record.get('extension', []):
        if extension['url'] == 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-race':
            patient_data['race'] = extension['extension'][0]['valueCoding']['display']
        elif extension['url'] == 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity':
            patient_data['ethnicity'] = extension['extension'][0]['valueCoding']['display']
        elif extension['url'] == 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex':
            patient_data['birthSex'] = extension['valueCode']

    return patient_data

# Function to parse a single observation record
def parse_observation_record(record):
    observation_data = {
        'id': record.get('id', None),
        'status': record.get('status', None),
        'category_code': record.get('category', [{}])[0].get('coding', [{}])[0].get('code', None),
        'test_code': record.get('code', {}).get('coding', [{}])[0].get('code', None),
        'test_name': record.get('code', {}).get('coding', [{}])[0].get('display', None),
        'patient_reference': record.get('subject', {}).get('reference', None),
        'encounter_reference': record.get('encounter', {}).get('reference', None),
        'effectiveDateTime': record.get('effectiveDateTime', None),
        'value': None,
        'unit': None
    }

    # Check if 'valueQuantity' exists before accessing it
    if 'valueQuantity' in record:
        observation_data['value'] = record['valueQuantity'].get('value', None)
        observation_data['unit'] = record['valueQuantity'].get('unit', None)

    return observation_data

def find_observations_for_patient(df, patient_ref):
    """
    Function to find all observation resources for a specific patient.

    Parameters:
    df (DataFrame): The DataFrame containing observation data.
    patient_ref (str): The patient reference to filter by.

    Returns:
    DataFrame: A DataFrame containing only the observations for the specified patient.
    """
    filtered_df = df[df['patient_reference'] == patient_ref]
    return filtered_df

specific_patient_ref = "Patient/9c3ebb7e-d087-519e-bea4-31c3d4aac7ff"
observations_for_patient = find_observations_for_patient(df, specific_patient_ref)

# To view the results
observations_for_patient

# Read the file for patients
patients = []
with open('Patient.ndjson', 'r') as file:
    for line in file:
        json_record = json.loads(line)
        patients.append(parse_patient_record(json_record))

# Reading the file for observations
observations = []
with open('ObservationLabevents.ndjson', 'r') as file:
    for line in file:
        json_record = json.loads(line)
        observations.append(parse_observation_record(json_record))

# Converting both lists to a dataframe
patient_df = pd.DataFrame(patients)
observation_df = pd.DataFrame(observations)

# Using the main web app

# Identify the patient of interest
specific_patient_ref = "Patient/9c3ebb7e-d087-519e-bea4-31c3d4aac7ff"

# Get all observations for that patient
observations_for_patient = find_observations_for_patient(observation_df, specific_patient_ref)

# Goals for turning this into a web app

- Upload the patient and lab files to github
- Organize all the functions into a single section in this colab notebook
- Remove any unnecessary artifacts in the notebook to keep it organized
- Send all the code to chatGPT to convert to streamlit so I can deploy it as a web app